In [1]:
import argparse
import numpy as np

from utils import *
from prompts import *
from tqdm import tqdm
from itertools import product
from openai import OpenAI
from os import path, makedirs
from datasets import load_dataset

def evaluate_gpt_4o_series(data, client, model):    
    model_outputs = []            
    for i in tqdm(range(len(data))):
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": data['text'][i]},
            ],
            temperature=0,
            max_tokens=10,
            top_p=1
        )
        model_outputs.append(response.choices[0].message.content)
    return model_outputs

In [2]:
#-------------------
# parameters
#-------------------    
parser = argparse.ArgumentParser()
parser.add_argument('--model_id', type=str, default='4o_mini')
parser.add_argument('--dataset', type=str, default='beanham/spatial_join_dataset')
parser.add_argument('--key', type=str, default='openaikey')
args = parser.parse_args(args=[])
args.save_path=f'inference_results/base/{args.model_id}_sc/'

In [3]:
def formatting_prompts_func(example):
    output = ""
    if 'value_angle' in config:
        input = "Sidewalk: "+str(example['sidewalk'])+"\nRoad: "+str(example['road'])+\
                "\nmin_angle: "+str(example['min_angle'])
    elif 'value_distance' in config:
        input = "Sidewalk: "+str(example['sidewalk'])+"\nRoad: "+str(example['road'])+\
                "\nmin_distance: "+str(example['euc_dist'])    
    elif 'value_comb' in config:
        input = "Sidewalk: "+str(example['sidewalk'])+"\nRoad: "+str(example['road'])+\
                "\nmin_angle: "+str(example['min_angle'])+"\nmin_distance: "+str(example['euc_dist'])        
    else:
        input = "Sidewalk: "+str(example['sidewalk'])+"\nRoad: "+str(example['road'])
    text = base_alpaca_prompt.format(base_instruction, input, output)
    return { "text" : text}

In [10]:
args.model_repo = MODEL_REPOS[args.model_id]
client = OpenAI(api_key="")
data = load_dataset(args.dataset)
config="zero_shot_with_heur_value_angle"
base_instruction=INSTRUCTIONS[config]
test = data['test'].map(formatting_prompts_func)
critique="\n\n### Review: \nPlease review and find if there are any problems with your response. Spebcifially, check if min_angle supports the parallelism condition."
improve="\n\n### Improve: \nBased on your review, please improve your response. \n\n### Response: "

In [11]:
current=np.load(f'inference_results/base/4o_mini/4o_mini_{config}.npy')
current=np.array([int(i) for i in current])
np.where(current!=np.array(test['label']))[0][:10]

array([ 27,  37,  66,  82,  96, 125, 165, 186, 193, 197])

In [21]:
index=37
print(test['text'][index])

### Instruction:
You are a helpful geospatial analysis assistant. I will provide you with a pair of (sidewalk, road) geometries in GeoJSON format, along with a key statistics:

- min_angle: The minimum angle (in degrees) between the sidewalk and the road.

Your task is to determine whether the sidewalk runs alongside the road by evaluating the following condition:

- Parallelism: The sidewalk should be approximately parallel to the road, with only a small angle difference between their orientations. The min_angle value provides a measure of this alignment.

If the condition is satisfied, return 1. Otherwise, return 0. No explaination is needed.

### Input:
Sidewalk: {'coordinates': [[-122.14894109999999, 47.615990700000005], [-122.14894500000001, 47.6160031], [-122.14894940000002, 47.6160102], [-122.1489589, 47.6160192], [-122.14897, 47.61602679999999], [-122.14898400000001, 47.61603309999998], [-122.14899680000002, 47.6160373], [-122.1490118, 47.616037999999996], [-122.1490319, 47.616

In [22]:
initial_response = client.chat.completions.create(
     model=args.model_repo,
     messages=[
         {"role": "user", "content": test['text'][index]},
     ],
     temperature=0,
     max_tokens=10,
     top_p=1,
)

In [24]:
initial_response.choices[0].message.content, test['label'][index]

('1', 0)

In [25]:
new_input=test['text'][index].strip()+initial_response.choices[0].message.content+critique
print(new_input)

### Instruction:
You are a helpful geospatial analysis assistant. I will provide you with a pair of (sidewalk, road) geometries in GeoJSON format, along with a key statistics:

- min_angle: The minimum angle (in degrees) between the sidewalk and the road.

Your task is to determine whether the sidewalk runs alongside the road by evaluating the following condition:

- Parallelism: The sidewalk should be approximately parallel to the road, with only a small angle difference between their orientations. The min_angle value provides a measure of this alignment.

If the condition is satisfied, return 1. Otherwise, return 0. No explaination is needed.

### Input:
Sidewalk: {'coordinates': [[-122.14894109999999, 47.615990700000005], [-122.14894500000001, 47.6160031], [-122.14894940000002, 47.6160102], [-122.1489589, 47.6160192], [-122.14897, 47.61602679999999], [-122.14898400000001, 47.61603309999998], [-122.14899680000002, 47.6160373], [-122.1490118, 47.616037999999996], [-122.1490319, 47.616

In [26]:
critique_response = client.chat.completions.create(
     model=args.model_repo,
     messages=[
         {"role": "user", "content": new_input},
     ],
     temperature=0,
     max_tokens=100,
     top_p=1
 )
critique_response.choices[0].message.content

'The provided min_angle is 15.238866212783577 degrees. Since this angle is greater than a typical threshold for parallelism (often around 10 degrees), it indicates that the sidewalk is not approximately parallel to the road. Therefore, the correct response should be:\n\n### Response: 0'

In [27]:
new_input=test['text'][index].strip()+initial_response.choices[0].message.content+critique+'\n\n'+critique_response.choices[0].message.content+improve
print(new_input)

### Instruction:
You are a helpful geospatial analysis assistant. I will provide you with a pair of (sidewalk, road) geometries in GeoJSON format, along with a key statistics:

- min_angle: The minimum angle (in degrees) between the sidewalk and the road.

Your task is to determine whether the sidewalk runs alongside the road by evaluating the following condition:

- Parallelism: The sidewalk should be approximately parallel to the road, with only a small angle difference between their orientations. The min_angle value provides a measure of this alignment.

If the condition is satisfied, return 1. Otherwise, return 0. No explaination is needed.

### Input:
Sidewalk: {'coordinates': [[-122.14894109999999, 47.615990700000005], [-122.14894500000001, 47.6160031], [-122.14894940000002, 47.6160102], [-122.1489589, 47.6160192], [-122.14897, 47.61602679999999], [-122.14898400000001, 47.61603309999998], [-122.14899680000002, 47.6160373], [-122.1490118, 47.616037999999996], [-122.1490319, 47.616

In [28]:
improve_response = client.chat.completions.create(
     model=args.model_repo,
     messages=[
         {"role": "user", "content": new_input},
     ],
     temperature=0,
     max_tokens=100,
     top_p=1
 )
improve_response.choices[0].message.content

'0'